# Change the ecoinvent version of the mapping file

In [1]:
import pandas as pd
import bw2data as bd
from mescal import *

In [2]:
esm_location = 'CA-QC' # can be 'CH' or 'CA-QC'
ei_version_from = '3.8'
ei_version_to = '3.10.1'

In [3]:
bd.projects.set_current(f"ecoinvent{ei_version_to}")

In [4]:
name_comp_db = f'ecoinvent_cutoff_{ei_version_from}_remind_SSP2-Base_2020_comp_{esm_location}'

In [5]:
mapping = pd.read_csv(f"energyscope_data/{esm_location}/mapping_{ei_version_from}_linked.csv")

In [6]:
def change_mobility_name(row):
    
    # Remove year
    row.Activity = row.Activity.replace(', 2020', '')

    # Remove EURO-6
    row.Product = row.Product.replace('truck, Euro-6', 'truck')
    row.Activity = row.Activity.replace('Euro-6, regional delivery', 'regional delivery')
    row.Activity = row.Activity.replace('Euro-6, urban delivery', 'urban delivery')
    
    # Battery electric vehicles
    row.Activity = row.Activity.replace('NMC-622 battery, ', '')
    
    return row

In [7]:
if ei_version_to in ['3.10', '3.10.1']:
    mapping = mapping.apply(change_mobility_name, axis=1) # name changes coming with carculator_truck v0.5.0

In [8]:
updated_mapping, unit_changes = change_ecoinvent_version_mapping(mapping, ei_version_from, ei_version_to, name_comp_db)

BATTERY Construction
Old: battery, Li-ion, rechargeable, prismatic - market for battery, Li-ion, rechargeable, prismatic - GLO
New: battery, Li-ion, LiMn2O4, rechargeable, prismatic - market for battery, Li-ion, LiMn2O4, rechargeable, prismatic - GLO
CEMENT_PROD Operation
Old: cement, alternative constituents 21-35% - cement production, alternative constituents 21-35% - RoW
New: cement, CEM II/B - cement production, CEM II/B - RoW
CEMENT_PROD_HP Operation
Old: cement, alternative constituents 21-35% - cement production, alternative constituents 21-35% - RoW
New: cement, CEM II/B - cement production, CEM II/B - RoW
CO2_METHANOL Operation
Old: methanol - methanol production - GLO
New: methanol - methanol production, natural gas reforming - CN
CUMENE_PROCESS Operation
Old: cumene - cumene production - RoW
New: cumene - cumene production, benzene alkylation - RoW
ELEC_STO Construction
Old: battery, Li-ion, rechargeable, prismatic - market for battery, Li-ion, rechargeable, prismatic - GLO


In [9]:
updated_mapping.head()

,Name,Type,Product,Activity,Location,Database
0,AFC,Construction,"fuel cell system, 1 kWe, proton exchange membr...","fuel cell system assembly, 1 kWe, proton excha...",GLO,ecoinvent_cutoff_3.10.1_remind_SSP2-Base_2020
1,AFC,Operation,"electricity, from residential heating system","electricity, residential, by conversion of hyd...",CH,ecoinvent_cutoff_3.10.1_remind_SSP2-Base_2020
2,ALKALINE_ELECTROLYSIS,Operation,"hydrogen, gaseous, 20 bar","hydrogen production, gaseous, 20 bar, from AEC...",CH,ecoinvent_cutoff_3.10.1_remind_SSP2-Base_2020
3,ALKALINE_ELECTROLYSIS_PLANT,Construction,"electrolyzer, 1MWe, AEC, Balance of Plant","electrolyzer production, 1MWe, AEC, Balance of...",RER,ecoinvent_cutoff_3.10.1_remind_SSP2-Base_2020
4,ALKALINE_ELECTROLYSIS_PLANT_DECOM,Construction,"used fuel cell balance of plant, 1MWe, AEC","treatment of fuel cell balance of plant, 1MWe,...",RER,ecoinvent_cutoff_3.10.1_remind_SSP2-Base_2020


In [10]:
updated_mapping.Database = f'ecoinvent_cutoff_{ei_version_to}_image_SSP2-Base_2020+truck_carculator'

In [11]:
updated_mapping.Database.unique()

array(['ecoinvent_cutoff_3.10.1_image_SSP2-Base_2020+truck_carculator'],
      dtype=object)

In [12]:
base_db = Database(db_names=list(updated_mapping.Database.unique()))
unlinked = base_db.test_mapping_file(updated_mapping)

Getting activity data


100%|██████████| 34983/34983 [00:00<00:00, 52913.82it/s]


Adding exchange data to activities


100%|██████████| 1009584/1009584 [01:02<00:00, 16166.73it/s]


Filling out exchange data


100%|██████████| 34983/34983 [00:03<00:00, 9056.32it/s] 


Loaded ecoinvent_cutoff_3.10.1_image_SSP2-Base_2020+truck_carculator from brightway!
Mapping successfully linked to the database


In [13]:
if len(unlinked) == 0:
    updated_mapping.to_csv(f"energyscope_data/{esm_location}/mapping_{ei_version_to}.csv", index=False)
else:
    print(f"Unlinked exchanges: {unlinked}")

# Adapt the unit conversion file corresponding to the new mapping file

In [14]:
# unit_conversion = pd.read_excel(f"energyscope_data/{esm_location}/unit_conversion_{ei_version_from}.xlsx")

In [15]:
# Allows to keep formulas in Excel files
from openpyxl import load_workbook

In [16]:
wb = load_workbook(filename=f"energyscope_data/{esm_location}/unit_conversion_{ei_version_from}.xlsx")
unit_conversion = pd.DataFrame(wb[wb.sheetnames[0]].values)

In [17]:
# setting first row as header
new_header = unit_conversion.iloc[0]
unit_conversion = unit_conversion[1:]
unit_conversion.columns = new_header

In [18]:
unit_changes

[[('OCGT_LARGE', 'Operation'),
  ('electricity, medium voltage',
   'natural gas, burned in gas turbine, for compressor station',
   'CA-QC'),
  'kilowatt hour',
  'megajoule'],
 [('OCGT_SMALL', 'Operation'),
  ('electricity, medium voltage',
   'natural gas, burned in gas turbine, for compressor station',
   'CA-QC'),
  'kilowatt hour',
  'megajoule']]

In [19]:
# Add new unit conversion factors
new_unit_conversion_factors = {
    ('OCGT_LARGE', 'Operation'): ['= 3.6 * 2.5', '3.6 MJ of heat per kWh, 2.5 kWh NG per kWh electricity'],
    ('OCGT_SMALL', 'Operation'): ['= 3.6 * 2.5', '3.6 MJ of heat per kWh, 2.5 kWh NG per kWh electricity'],
}

In [20]:
unit_conversion = update_unit_conversion_file(unit_conversion, unit_changes, new_unit_conversion_factors)

In [21]:
unit_conversion.sort_values(by=['Name', 'Type']).to_excel(f"energyscope_data/{esm_location}/unit_conversion_{ei_version_to}.xlsx", index=False)